End ALS challenge submission - Ryan Cloke, ryan.cloke@gmail.com

**Executive Summary**

Task 1: machine learning identified two key interesting mutations: rs577067524 impacting a metalloproteinase with known involvement in neuroinflammation, and rs750805176 impacting SEPTIN2 which has been associated with neurofibrilary tangles in CNS disease.  These SNPs have no known clinical significance and should be investigated further. 

Task 2: machine learning indentified GDF6 and AC096719.1 as key predictive features.  AC096719.1 is a novel transcript and should be studied. Limb origination cases show statistically lower GDF6 expression and perhaps this subset of cases could benefit from growth factor supplementation.

Task 3: machine learning identified two proteins that have been previously mentioned as having ALS involvement in the literature.  APEH is also involved in protein degradation and AOX3 has been suggested in previous reports as a possible target for therapeutic intervention.

In conclusion, this dataset contains some interesting insights into ALS etiology and pathology.  We should be careful about model overfitting with only ~130 patients and >180,000 features in these joined datasets, but efforts were taken to reduce overfitting.  Ultimately increasing the number of participants in the study and adding additional omic information (methylation and microbiome) should uncover new therapeutic targets and strategies.

First, build the data processing pipeline (random forest, pseudo-gwas, rna comparison functions)

In [ ]:
#NOTE: give this about 2 minutes to run

#import the modules needed for this pipeline
import numpy as np 
import pandas as pd 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from yellowbrick.target import ClassBalance
import imblearn
from sklearn.metrics import roc_auc_score
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

#load and join data files on participant_id
df_rna = pd.read_csv('/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv')

df_stain = pd.read_csv('/kaggle/input/end-als/end-als/clinical-data/filtered-metadata/metadata/clinical/Answer ALS metadata datatable with QC Staining.csv')
df_stain = df_stain[['GUID','NEFH (SMI32)','ISL1','NKX6.1','TUBB3 (TuJ1)','s100b','Nestin']]
df_stain = df_stain.rename(columns = {'GUID':'Participant_ID'})

df_genomics = pd.read_csv('/kaggle/input/end-als/end-als/genomics-data/geno_bin.csv',index_col = [0]).T
df_genomics.reset_index(inplace=True)
df_genomics = df_genomics.rename(columns = {'index':'unformatted_ID'})
df_genomics['Participant_ID'] = df_genomics.unformatted_ID.apply(lambda x: x[5:])
df_genomics = df_genomics.drop(columns=['unformatted_ID'])

df_mergeone = pd.merge(df_rna, df_stain, on='Participant_ID', how='inner')
df_mergeone.to_csv('/kaggle/working/stain_rna.csv')

#dataset for random forest
df_merged = pd.merge(df_mergeone, df_genomics, on='Participant_ID', how='inner')
df_merged.to_csv('/kaggle/working/stain_genomics_rna.csv')

#dataset for gwas
df_labels = df_rna[['Participant_ID','CtrlVsCase_Classifier']]
df_gwas = pd.merge(df_labels, df_genomics, on='Participant_ID', how='inner')

In [ ]:
###Build a random forest machine learning model####

def rfClassifier(dataset,do_SMOTE,feature_idx,label_idx,ntrees):
    #fillNA, split dataset into features and labels for supervised learning
    df = dataset.fillna(0)
    
    X, y = df.iloc[:,feature_idx:],df.iloc[:,label_idx]

    #standardize the dataset for ML
    X_standard = StandardScaler().fit_transform(X)
    X = pd.DataFrame(X_standard, columns=X.columns)

    #split the dataset randomly into train and test with 20% test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  
    
    #Check the class balance
    visualizer = ClassBalance(labels=["class 0","class 1"])
    visualizer.fit(y)        
    visualizer.show()        

    #If the ctrl and case numbers are imbalanced the model overtrain on one class and predict all 1 for example
    #rebalance the dataset by oversampling with synthetic examples using SMOTE
    if do_SMOTE == True:
        oversample = imblearn.over_sampling.SMOTE()

        # fit and apply the transform
        X_over, y_over = oversample.fit_resample(X_train, y_train)
        
    else:
        X_over, y_over = X_train, y_train
        
    print('oversampled shape of X is ',X_over.shape)
    print('oversampled shape of y is ',y_over.shape,'\n')

    #save the feature names for later
    features = X_over.columns.tolist()

    #define a RF classifier object and fit a model
    random_forest = RandomForestClassifier(n_estimators=ntrees,random_state=123)
    random_forest.fit(X_train,y_train)

    #rank the features and store in a dataframe
    feature_importance_values = random_forest.feature_importances_
    feature_importances = pd.DataFrame({'feature': features, 'importance':feature_importance_values})

    #create a new dataframe called newX containing only features that have some predictive value
    #this is done in order to avoid overfitting (which will certainly happen with 180k features)
    key_features = feature_importances[feature_importances.importance != 0.0]
    key_features = key_features.sort_values(by='importance',ascending=False)
    key_features.to_csv('/kaggle/working/key_features_dna_rna.csv')
    key_col_names = key_features["feature"].tolist()
    newXtrain = X_over[X_over.columns.intersection(key_col_names)]
    newXtest = X_test[X_over.columns.intersection(key_col_names)]

    print('reduced dimension of training input from ', X_train.shape, 'to ', newXtrain.shape, 'by dropping irrelevant features \n')

    #plot top 20 key feature importance
    key_features.head(20).plot(kind='barh',x='feature',y='importance',color='blue')
    plt.show()

    #train a new model using the input with reduced dimensionality and make predictions
    random_forest.fit(newXtrain,y_over)
    preds = random_forest.predict(newXtest)
    predictions = [round(value) for value in preds]
    print('Check that the output prediction is not simply returning a trivial solution of all 1s \n',predictions,'\n')

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Simple accuracy is: %.2f%%" % (accuracy * 100.0),'\n')
    print('ROC AUC is ', roc_auc_score(y_test, predictions),'\n')

    explainer = shap.TreeExplainer(random_forest)
    shap_values = explainer.shap_values(newXtrain)
    shap.summary_plot(shap_values[1], newXtrain)


In [ ]:
###Create a function to do a pseudo-GWAS and visualize the SNPs with the biggest discrepancies between classes###

def pseudo_gwas(dataset,label_name,feature_idx):
    featureLst=dataset.columns.tolist()
    featureLst = featureLst[feature_idx:]
    print('checking ',len(featureLst), ' SNP variations and comparing between classes\n')
    counter=0
    case_lst=[]
    ctrl_lst=[]

    #calculate sample size
    total_patients = dataset.shape[0]
    case_patients = dataset.loc[dataset[label_name]>0].shape[0]
    ctrl_patients = total_patients - case_patients

    #fillna
    dataset = dataset.fillna(0)

    #check each SNP and compute the difference in frequency between classes
    for feature in featureLst:
        #sum num mutations for each patient group for each feature
        var_sum = dataset.groupby(by=[label_name])[feature].sum()
        ctrl_sum = var_sum.iloc[[0]]
        case_sum = var_sum.iloc[[1]]
        case_sum = float(case_sum.values)
        ctrl_sum = float(ctrl_sum.values)
        case_pct = (case_sum / case_patients)*100
        ctrl_pct = (ctrl_sum / ctrl_patients)*100

        #save the frequency of SNP per class for plotting later
        case_lst.append(case_pct)
        ctrl_lst.append(ctrl_pct)
        
        #print progress
        counter = counter+1
        if counter%10000 == 0:
            print('checked ', counter, ' SNPs so far')
            
    #make a new dataframe for plotting and analysis
    d = {'case_pct': case_lst, 'ctrl_pct': ctrl_lst}
    df_var = pd.DataFrame(data=d, index=featureLst)
    df_var['delta'] = df_var['case_pct'] - df_var['ctrl_pct']
    df_var = df_var.sort_values(by='delta',ascending=False)

    ax = df_var.head(10).plot.bar(rot=0,figsize=(20,20))
    ax.set_xlabel("key variant identified by ML")
    ax.set_ylabel("Percent of participants with that variant")

    df_var.to_csv('/kaggle/working/gwas.csv')

In [ ]:
###Create a similar function but for RNA seq and compare discrepancies between classes###

def rna_comparison(dataset,label_name,feature_idx):
    featureLst=dataset.columns.tolist()
    featureLst = featureLst[feature_idx:]
    print('checking ',len(featureLst), ' gene expression levels and comparing between classes\n')
    counter=0
    case_lst=[]
    ctrl_lst=[]

    #calculate sample size
    total_patients = dataset.shape[0]
    case_patients = dataset.loc[dataset[label_name]>0].shape[0]
    ctrl_patients = total_patients - case_patients

    #normalize data since rna value is continuous instead of binary
    df_gwas_rna = dataset
    df_gwas_rna = df_gwas_rna.fillna(0)
    df_gwas_rna = df_gwas_rna.drop(columns=['Participant_ID'])
    x_scaled = MinMaxScaler().fit_transform(df_gwas_rna)
    df_gwas_rna = pd.DataFrame(x_scaled, columns=df_gwas_rna.columns)

    #check each gene and compute the difference in frequency between classes
    for feature in featureLst:
        #sum gene expression for each patient group for each feature
        var_sum = df_gwas_rna.groupby(by=[label_name])[feature].sum()
        ctrl_sum = var_sum.iloc[[0]]
        case_sum = var_sum.iloc[[1]]
        case_sum = float(case_sum.values)
        ctrl_sum = float(ctrl_sum.values)
        case_pct = (case_sum / case_patients)
        ctrl_pct = (ctrl_sum / ctrl_patients)

        #save the frequency of gene expression for plotting and analysis later
        case_lst.append(case_pct)
        ctrl_lst.append(ctrl_pct)
        
        #track progress
        counter = counter+1
        if counter%10000 == 0:
            print('checked ', counter, ' genes so far')
    
    ##make a new dataframe for plotting and analysis
    d = {'case_pct': case_lst, 'ctrl_pct': ctrl_lst}
    df_var = pd.DataFrame(data=d, index=featureLst)
    df_var['delta'] = df_var['case_pct'] - df_var['ctrl_pct']
    df_var = df_var.sort_values(by='delta',ascending=False)

    ax = df_var.head(10).plot.bar(rot=0,figsize=(20,20))
    ax.set_xlabel("Gene")
    ax.set_ylabel("Average relative expression of genes for top 10 differences between classes")
    
    #also plot which genes are not expressed by case=1 but are expressed by case=0
    df_no_expression = df_var[(df_var['case_pct']==0)&(df_var['ctrl_pct']>0.07)]
    ax = df_no_expression.head(10).plot.bar(rot=0,figsize=(20,20))
    ax.set_xlabel("Gene")
    ax.set_ylabel("Average relative expression for genes not expressed by class = 1 but expressed by class =0")

# Task #1: Does ALS have one mechanism of action (one pathway) or is it caused by multiple independent or different mechanisms of action (multiple pathways)?

In [ ]:
#step 1a - run the random forest classifier
rfClassifier(dataset=df_merged,do_SMOTE=True,feature_idx=2,label_idx=1,ntrees=1000)

#step 1b - run the pseudo-gwas
pseudo_gwas(dataset=df_gwas,label_name = 'CtrlVsCase_Classifier',feature_idx = 2)

#step 1c - run the rna comparison function
rna_comparison(dataset=df_rna,label_name = 'CtrlVsCase_Classifier',feature_idx = 2)

# Task 1 Discussion.

I used a random forest classifier after balancing and normalizing the input data.  I used the SMOTE implementation to upsample the ctrl cases.  Without balancing the classes, the model makes only trivial predictions with the output vector being all "1" class.  By balancing example numbers of case and ctrl (real + synthetic), the model achieved improved ROC AUC.  

The high accuracy of the model tells us that there is good information in this dataset that should inform us about the underlying causes of ALS.  However, looking at the feature importance, it is clear that there are many features with roughly equal importance, indicating that ALS is a complicated interaction between many SNPs and gene expression levels.  I looked through the features with high importance in RF feature ranking and SHAP plot, and summarized key findings in the table below.

| SNP|GENES|biochemical pathways|clinVar|known association with CNS disease|
|---|---|---|---|---|
| rs750805176  | SEPTIN2  |microtubles and filaments   |not reported in ClinVar   |Yes, associated with neurofibrillary tangles, pathology in Alzheimer's    |
| rs577067524  | MMP24  |ECM breakdown and remodeling   |not reported in ClinVar   |Yes, controls neuroinflammation.  Known Tx target for AD/ALS/PD/HD/MS   |
| chr7:111801687  | DOCK4  |cytokinesis and adheren regulation |NA |mutations a topic of research in ALZ/ALS |


**rs577067524** - this indel mutation impacts MMP24, a metalloproteinase involved in neuroinflammation. MMPs are a prime suspect in ALS and are considered a potential therapeutic target (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4619970/). Considering the known role of MMPs in neurodegeneration, this SNP should be looked at closely as there are no publications or clinvar reports in dbSNP.  The mutated protein should be synthesized and we should study whether this mutant has any altered enzymatic function.

**rs750805176** - this frameshift mutation impacts SEPTIN2 coding. Knowndown of SEPTIN2 has been shown to cause actin fiber disintegration and cell polarity loss (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2085444/). SEPTIN2 has also been associated with neurofibrillary tangles in Alzheimer's disease (https://www.sciencedirect.com/science/article/pii/S0300908411003609). Since actin and microtuble pathways are suspect in ALS, this mutation should be looked at closely.

**chr7:111801687** - mutation in a part of the genome coding for DOCK4 and a member of Guanine Exchange Factors (GEF) family which have been a topic of active interest in Alzheimer's and ALS (https://www.frontiersin.org/articles/10.3389/fncel.2014.00282/full).

Next, I wanted to compare the distribution of SNPs between case and control classes and check if any of the suspect SNPs above show a strong discrepancy between classes.  I calculated the percent of participants per class with that particular SNP and plotted the top 10 SNPs with the biggest delta between case/control and summarized in the table below.

| SNP|observed in ctrl|GENES|biochemical pathways|clinVar|known association with CNS disease|
|---|---|---|---|---|---|
| rs750805176  | yes|SEPTIN2  |microtubles and filaments   |not reported in ClinVar   |Yes, associated with neurofibrillary tangles, pathology in Alzheimer's    |
| chr7:111801687  | no|DOCK4|cytokinesis and adheren regulation    | NA  | mutations a topic of research in ALZ/ALS  |
| rs757676742  |yes|UBXN11|actin pathways|not reported in ClinVar | no  |
|chr19:42321227 | no|TMEM145 |GPCR | NA  | no  |
| chr6:3289981  | yes|SLC22A3,PSMG4 |transmembrane, assembly chaperone|NA|no  |
| chr1:61918051  | no|PATJ |mediates protein-protein interactions|NA|no  |
| rs561262047  | yes|ZNF714 |transcriptional regulation |not reported in ClinVar | no |
| chr15:43783008  |yes|SERF2|protein-protein interactions including ubiquitin|NA|Yes, speculated to be involved in ubiquitin pathways in AD|
| chr12:121240525  |no|CAMKK2|regulates neuropeptide Y|NA| no |
| chr3:50118454  | no |unknown |unknown |unknown  |


Sure enough, **rs750805176** and **chr7:111801687** show a large difference between case/control abundance.   Most of the remaining SNPs identified with this method do not have any known association with CNS disease, with the exception of chr15:43783008 / SERF2.  SERF2 plays an important role in the ubiquitin pathway, and knockout of this gene led to accumulation of toxic plaques in ALZ models (https://www.nature.com/articles/s41598-018-33143-1).

I also compared RNA seq expression levels between control and case and similarly plotted the top 10 genes with the largest difference in expression.  The top genes (233864,USP9Y,ZFY,PRKY,DDX3Y) were all located on the Y chromosome or associated with male fertility.  This suggests that being male is a risk factor for ALS.  In the future, we should filter proteins expressed in only one sex.

In summary, a combination of machine learning and pseudo-gwas revealed four interesting mutations that require additional study. SNPs in the coding regions for SEPTIN2, MMP24, DOCK4, and SERF2 were identified as very significant by this approach even though they have no known clinical significance in clinVar. Given that these genes all have plausible roles in ALS, we should test if these specific mutations result in loss of function. 

# **Task #2. What are potential mechanisms of disease progression?**


In [ ]:
#step 2a - run the random forest classifier
df_BL = pd.read_csv('/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv')
df_task2a = pd.merge(df_BL,df_genomics,on='Participant_ID',how='inner')
rfClassifier(dataset=df_task2a,do_SMOTE=True,feature_idx=2,label_idx=1,ntrees=1000)

#step 2b - run the pseudo-gwas
df_labels = df_BL[['Participant_ID','SiteOnset_Class']]
df_gwas = pd.merge(df_labels, df_genomics, on='Participant_ID', how='inner')
pseudo_gwas(dataset=df_gwas,label_name = 'SiteOnset_Class',feature_idx = 2)

#step 2b - run the rna comparison function
rna_comparison(dataset=df_BL,label_name = 'SiteOnset_Class',feature_idx = 2)

Using the same pipeline as in Task 1, I identified key gene expression patterns and SNPs shown in the table below.

|GENES|biochemical pathways|known association with CNS disease|reference|
|---|---|---|---|
|AC096719.1  |novel transcript|unknown|NA |
|GDF6 |growth factor| can bind directly to noggin, inhibiting neural growth / differentiation  | https://pubmed.ncbi.nlm.nih.gov/10393114/ |
|ELOCP21 |Transcription Elongation Factor B  |unknown |NA|
|AL163195.2  |novel transcript, antisense to RNASE11 & RNASE12  | unknown  |NA  |
|RASSF5   |suppresses cell growth in response to activated Rap1 or Ras |unknown |NA |
|AL118505.1|novel transcript |upregulation associated with better prognosis in glioma |https://pubmed.ncbi.nlm.nih.gov/30762207/|
|rs35821928 / LRP1D |LDL receptor related protein 1B |unknown |NA|
| FOXL2NB aka C3orf72  |transcription factor |known association of C3orf72 to ALS |https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6437781/|
| rs41285308 / SAMD3 |protein-protein interactions |possibly subunit of SARM1, a known target for ALS |https://www.cell.com/trends/pharmacological-sciences/fulltext/S0165-6147(20)30006-7|


**GDF6** - this is a growth factor that may be underexpressed in cases of ALS originating in the limbs (per the SHAP plot).  Aside from the atrophy that is a hallmark of ALS, GDF6 can also bind directly to and inhibit noggin which is an essential protein for neural growth and differentiation. 

**FOXL2NB aka C3orf72** - this is a transcription factor that may be underexpressed in cases of ALS originating in the limbs (per the SHAP plot).  It's function is not well understood but it has been previously associated with ALS.

**rs41285308 / SAMD3** - this SNP is in the coding region of SAMD3, a sterile alpha motif protein.  A protein called SARM1 is key in axon degeneration pathways, and this protein contains a SAM subunit.  Perhaps this mutation creates some loss of function in SAM proteins.

Let's check if the difference in GDF6 expression levels are statistically significant below.

In [ ]:
from scipy import stats
df_BL = pd.read_csv('/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv')
df_BL.boxplot(column=['GDF6'],by='SiteOnset_Class')

group1 = df_BL.where(df_BL.SiteOnset_Class== 0).dropna()['GDF6']
group2 = df_BL.where(df_BL.SiteOnset_Class== 1).dropna()['GDF6']

print('Lower GDF6 expression levels were found for limb onset with p value ', stats.ttest_ind(group1,group2)[1])

In general for this dataset of bulbar vs limb, protein expression levels are more important features than SNPs.  As all participants in this dataset have ALS, this could make sense due to common underlying genomic pathways giving rise to ALS, but differences in gene expression leading to changes in pathology.  It is recommended to understand the role of GDF6 better in ALS pathology and determine if certain patient subsets could benefit from supplemental growth factor therapy.

# **Task #3. What is the difference between people who progress fast vs slow?**

Do some dataset formatting and joining.  First create a dataframe with max FRS loss score (df_frs_delta).  Then create a dataset only with patients showing very rapid or relatively slow degeneration (df_survival).

In [ ]:
###create one dataset for random forest#####
#load the clinical data into memory and create a sum column for frs scores
df_frs = pd.read_csv("/kaggle//input/end-als/end-als/clinical-data/filtered-metadata/metadata/clinical/ALSFRS_R.csv")
df_frs['functional_sum']  = df_frs['alsfrs1']+df_frs['alsfrs2']+df_frs['alsfrs3']+df_frs['alsfrs4']+df_frs['alsfrs5']+df_frs['alsfrs6']+df_frs['alsfrs7']+df_frs['alsfrs8']+df_frs['alsfrs9']

#there are multiple rows for each patient, so calculate the maximum frs score change and date and use that for the unique participant row
df_frs_summary = df_frs[['SubjectUID','Visit_Date','functional_sum']]
df_min = df_frs_summary.groupby(by=['SubjectUID'],as_index=False)['functional_sum'].min()
df_min_date = df_frs_summary.merge(df_min, on=['SubjectUID','functional_sum'],how='inner')
df_min_date['min_functional_sum'] = df_min_date['functional_sum']
df_min_date = df_min_date.drop(columns='functional_sum')
df_initial = df_frs_summary[df_frs_summary['Visit_Date']==0]
df_initial['initial_functional_sum']  = df_initial['functional_sum']
df_initial =df_initial.drop(columns = ['functional_sum','Visit_Date'])

df_frs_delta = pd.merge(df_initial, df_min_date, on='SubjectUID',how='inner')

#drop multiple subjectIDs where score didn't change between visits
df_frs_delta  = df_frs_delta.drop_duplicates(['SubjectUID', 'min_functional_sum'])
df_frs_delta['delta_functional_score'] = df_frs_delta['initial_functional_sum'] - df_frs_delta['min_functional_sum']

#calculate a score for how rapidly frs changes
df_frs_delta['rapid_onset_score'] = df_frs_delta['delta_functional_score'] / df_frs_delta['Visit_Date']

#plot the data
df_frs_delta.hist(column='rapid_onset_score',bins=30)
df_frs_delta.plot(kind='scatter',x='Visit_Date',y='delta_functional_score')
print('mean rapid onset score is ', np.mean(df_frs_delta['rapid_onset_score']))
df_frs_delta['rapid_onset_class'] = np.where(df_frs_delta['rapid_onset_score'] >= np.mean(df_frs_delta['rapid_onset_score']), 1, 0)
df_frs_delta = df_frs_delta.rename(columns = {'SubjectUID':'Participant_ID'})

df_frs_delta = df_frs_delta.rename(columns = {'SubjectUID':'Participant_ID'})
df_onset_rna = pd.merge(df_frs_delta,df_rna, on='Participant_ID',how='inner')

###test a second approach where I create a cohort for long term and short term survival (frs score loss)
###create cohort with long term survival
df_frs_lts = df_frs_delta[(df_frs_delta['delta_functional_score']<5)&(df_frs_delta['Visit_Date']>(365*2))]
df_frs_lts['survival_prospect'] = 1

#create cohort with short term survival
df_frs_sts = df_frs_delta[(df_frs_delta['delta_functional_score']>10)&(df_frs_delta['Visit_Date']<(365*1))]
df_frs_sts['survival_prospect'] = 0

#stack dataframes and create new classification column for short or long term survival
df_survival = pd.concat([df_frs_lts,df_frs_sts])
df_survival = df_survival.rename(columns = {'SubjectUID':'Participant_ID'})

#join to transcriptomics data
df_survival_rna = pd.merge(df_survival,df_rna, on='Participant_ID',how='inner')


Distribution centered around 0.02 FRS loss per day, or 1 unit of FRS loss every 50 days, with some outliers showing much slower disease progression. Now try to do machine learning on this rapid onset score.

In [ ]:
rfClassifier(dataset=df_onset_rna,do_SMOTE=True,feature_idx=8,label_idx=6,ntrees=1000)


Random forest approach to predicting rapid vs delayed onset in ALS achieve a test set prediction of ~97%.  Key findings from the RF feature ranking and SHAP plots are listed below:

|GENES|biochemical pathways|known association with CNS disease|reference|
|---|---|---|---|
|HNRNPA3P4  |ribonuclear protein |NA | NA|
|AC073621.1 |ubiquitin protein ligase pseudogene | NA  | NA  |
|STRA8 |transcriptional factor |NA |NA|
| APEH |protein degradation  | yes, previously identified as associated with ALS  |https://iris.unimore.it/retrieve/handle/11380/1160039/238225/1-s2.0-S0160412018300643-main.pdf  |
|AOX3P |aldehyde oxidase | yes, possible target for ALS intervention| https://www.researchgate.net/profile/Yixue-Qiao/publication/337789029_Inhibition_of_vertebrate_aldehyde_oxidase_as_a_therapeutic_treatment_for_cancer_obesity_aging_and_amyotrophic_lateral_sclerosis/links/5f98b7cf92851c14bced3a01/Inhibition-of-vertebrate-aldehyde-oxidase-as-a-therapeutic-treatment-for-cancer-obesity-aging-and-amyotrophic-lateral-sclerosis.pdf | 


AC073621.1 could be interesting to follow up on.  It has no reports as being associated with ALS, but is involved in ubiquitin pathways which have been implicated in ALS.  Two other proteins with known association to ALS were identified by this method: APEH and AOX3P.  APEH is also involved in protein degradation and AOX3 has been suggested in previous reports as a possible target for therapeutic intervention.

In [ ]:
###try to rank features on the tiny dataset with extreme examples of rapid onset vs delayed frs score loss
from sklearn.ensemble import ExtraTreesClassifier

# define dataset
X, y = df_survival_rna.iloc[:,8:],df_survival_rna.iloc[:,6]

#save the feature names for later
feature_names = X.columns.tolist()

#https://www.kaggle.com/rafjaa/dealing-with-very-small-datasets
TOP_FEATURES = 5

forest = ExtraTreesClassifier(n_estimators=1000, max_depth=5, random_state=1)
forest.fit(X, y)

importances = forest.feature_importances_
std = np.std(
    [tree.feature_importances_ for tree in forest.estimators_],
    axis=0
)
indices = np.argsort(importances)[::-1]
indices = indices[:TOP_FEATURES]

print('Top features:')
for f in range(TOP_FEATURES):
    print(f + 1, feature_names[indices[f]], importances[indices[f]])


I like this approach in general of comparing the extreme examples in the patient population, but unfortunately there are only 11 patients in this dataset, and it makes machine learning challenging.

Overall, random forest on the larger patient population provided more insight into possible reasons for delayed vs rapid functional score loss.